In [ ]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import numpy as np
import sqlite3
import urllib
import urllib.request
import re
import datetime
import logging
import time
from selenium import webdriver

In [ ]:
now = datetime.datetime.now()
today = str(now.date())

In [ ]:
logging.basicConfig(filename='SE' + today + '.log', level=logging.DEBUG, format='%(asctime)s:%(name)s:%(message)s')

In [ ]:
url = 'https://www.arbetsformedlingen.se/For-arbetssokande/Hitta-jobb/pb-beta/home/search?cyo=3&sort=d&p=1'

# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)
        

In [ ]:
#decleare empty dataframe to store everything related to one role
role_dataframe_all = pd.DataFrame()
page_number = 1

In [ ]:
#iterate over job type list with dynamic url
#for each job iterate through all page to get all positions and store in dataframe, store: position name, company name, location
print(datetime.datetime.now())

try:

    while page_number < 50:
        
        print('waiting 5 sec')
        time.sleep(5)
    
        print(page_number)
        #print('connecting to : {}'.format(str(page_number)), datetime.datetime.now())
        
        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        job_title_results = soup.find_all('div', attrs={'class':'result-card-list-contianer'}) 
 

        #iterate over certain page to store position, company, and location in their own list

        pos_all = []
        for result in job_title_results:
            position = result.find('a').text
            job_description_url = result.find('a').get('href')
            company = result.find('b').text


            pos_all.append([position, job_description_url, company])

        #creating merged dataframe
        role_dataframe_page = pd.DataFrame(pos_all, columns=['position', 'url', 'company'])

        #adding data to role related dataframe
        role_dataframe_all = role_dataframe_all.append(role_dataframe_page, ignore_index=True)
        #increment page_number
        #print(now)
        #print(datetime.datetime.now())
        page_number += 1
        print('waiting 5 seconds')
        time.sleep(5)
        
        python_button = driver.find_element_by_class_name('next-button')
        driver.execute_script("arguments[0].click();", python_button)
        
        
except Exception as e:
    logging.exception("message")
    print("Error detected and logged")
    
print(datetime.datetime.now())

In [ ]:
role_dataframe_all.drop_duplicates(inplace=True)

In [ ]:
role_dataframe_all.to_excel('SE_IT_' + today+ '.xlsx')

In [ ]:
driver.close()

In [ ]:
start_url = role_dataframe_all['url'].iloc[0]

In [ ]:
start_url

In [ ]:
url = 'https://www.arbetsformedlingen.se' + start_url

# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

In [ ]:
def description_extractor(url):
    try:
        print(url)
        driver.get('https://www.arbetsformedlingen.se' + url)
        
        print('wait 5 seconds')
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        try:
            job_desc_results = soup.find('p', attrs={'class':'rekryteringsbehov-text break-pre-wrap ng-binding'}).text
        except Exception as e:
            job_desc_results = soup.find('div', attrs={'class':'annonstext-container'}).text

        job_desc_results = re.sub('\n', '', job_desc_results.strip())
        print('wait 5 sec')
        time.sleep(5)
        return job_desc_results
    except Exception as e:
        logging.exception("message")
        print("Error detected and logged")

In [ ]:
role_dataframe_all['description'] = role_dataframe_all['url'].apply(description_extractor)

In [ ]:
role_dataframe_all.to_excel('SE_FULL' + today+ '.xlsx')